#CBDE Lab 4 (part 1) - The Hadoop ecosystem (MapReduce)

#Assignment

# Setup environment

Install dependencies

In [ ]:
!pip3 install pyspark
import pyspark
from pyspark.sql import SparkSession
from pprint import pprint
import random
!pip3 install shortuuid
import shortuuid

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=e2dbb6bc7d5ae1034f35d83af4d8a49beed50964a5d28332abb4ef598185d58d
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


# Dataset description

We will use the same dataset as the one used in the HBase session. Please, refer to the HBase assignment for details on its structure.

In [ ]:
!wget --no-check-certificate https://archive.ics.uci.edu/static/public/2/adult.zip
!unzip adult.zip

--2023-11-13 17:45:48--  https://archive.ics.uci.edu/static/public/2/adult.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
  Issued certificate has expired.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘adult.zip’

adult.zip               [  <=>               ] 605.70K  1.61MB/s    in 0.4s    

2023-11-13 17:45:48 (1.61 MB/s) - ‘adult.zip’ saved [620237]

Archive:  adult.zip
  inflating: Index                   
  inflating: adult.data              
  inflating: adult.names             
  inflating: adult.test              
  inflating: old.adult.names         


In [ ]:
schema = ['id','age','workclass','fnlwgt','education','education-num','marital-status','occupation','relationship','race','sex','capital-gain','capital-loss','hours-per-week','native-country','income']
def getAttribute(row,att):
  for i,v in enumerate(schema):
    if (v == att):
      return row[i]
  return None

def executeMapReduce(mapFunction, reduceFunction):
    sc = pyspark.SparkContext.getOrCreate()
    pprint(sc.textFile("adult.data").map(lambda t: (str(shortuuid.uuid()),t.replace(' ',''))).filter(lambda t: len((t[0]+","+t[1]).split(","))==16).flatMap(lambda t: mapFunction(t[0],t[1])).groupByKey().flatMap(lambda t: reduceFunction(t[0],t[1])).take(25))

Here, we provide you with the method `getAttribute(array,attribute)`, which returns the projection for a specific attribute in the array. Note that the array should also contain the key as the first value (see the provided example).

# Exercise 1) Selection

Implement the selection operator. In SQL it would correspond to the following query:

```sql
SELECT * FROM adult WHERE workclass = "Private"
```

In [ ]:
def selection_map(k,v):
    # Concatenem la clau i el valor per a crear un string que representi la fila
    tupl = k + "," + v
    if "Private" in getAttribute(tupl.split(","), "workclass"): #comprovem que workclass sigui ""
      return[(tupl,1)] # Retornem les files que passin la condició com a key
    else:
       return []
def selection_reduce(k, lv):
    return [k] # Retornem la llista de keys, que es on hem guardat les files que passen la condició

executeMapReduce(selection_map,selection_reduce)

['VoDmkSoEWP6w76fwVzPmkA,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K',
 'QhES3rJFzcAdaERXMFZu7z,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K',
 '6Tmm5r4CgJyog2GjXKLcJk,49,Private,160187,9th,5,Married-spouse-absent,Other-service,Not-in-family,Black,Female,0,0,16,Jamaica,<=50K',
 'eFKzCVyZP7ERhz8hHJj9QP,42,Private,159449,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,5178,0,40,United-States,>50K',
 'HnK6Q5GYik3Ei7rQW49eqE,23,Private,122272,Bachelors,13,Never-married,Adm-clerical,Own-child,White,Female,0,0,30,United-States,<=50K',
 'ExGqoDNJr92PB4a2LCKipd,40,Private,121772,Assoc-voc,11,Married-civ-spouse,Craft-repair,Husband,Asian-Pac-Islander,Male,0,0,40,?,>50K',
 'BfMv5Fb4KYbUK4z58aDf9v,34,Private,245487,7th-8th,4,Married-civ-spouse,Transport-moving,Husband,Amer-Indian-Eskimo,Male,0,0,45,Mexico,<=50K',
 'KPexPnEiDKdFAy9XUUL3SP,40,Private,193524,Doct

# Exercise 2) Aggregation

Implement the aggregation operator (note this operation does not exist in standard SQL). In SQL it would correspond to the following query:

```sql
SELECT native_country, SUM(capital-gain) FROM adult GROUP BY native-country
```

In [ ]:
def aggregation_map(k,v):

    tupl = k + "," + v

    # Extreiem els atributs 'native-country' i 'capital-gain'
    native_country = getAttribute(tupl.split(","), "native-country")
    capital_gain = getAttribute(tupl.split(","), "capital-gain")

    # Retornem una parella  key-value amb clau 'native-country' i valor 'capital-gain' com a int per poder fer la suma
    return [(native_country, int(capital_gain))]
    # Amb aquesta solució estem retornant també les files que tenen 'native-country' null
    # En cas que la solució correcta sigui sense aquests valors, comprovariem abans del return que native-country != ?

def aggregation_reduce(k, lv):
    # Sumem els valors de 'capital-gain' de cada 'native-country'
    total_gain = sum(lv)

    # Retornem una llista amb les parelles 'native-country' i el 'capital-gain' total
    return [(k, total_gain)]

executeMapReduce(aggregation_map,aggregation_reduce)

[('United-States', 31772837),
 ('India', 360492),
 ('?', 1053200),
 ('Mexico', 223139),
 ('Honduras', 0),
 ('Iran', 87920),
 ('Philippines', 201632),
 ('Poland', 41057),
 ('Columbia', 10656),
 ('Cambodia', 19529),
 ('Thailand', 0),
 ('Ecuador', 14773),
 ('Laos', 2885),
 ('Portugal', 8450),
 ('France', 15864),
 ('Guatemala', 14773),
 ('China', 27486),
 ('Peru', 1831),
 ('Outlying-US(Guam-USVI-etc)', 0),
 ('Scotland', 5178),
 ('Greece', 37233),
 ('Vietnam', 44260),
 ('Hong', 0),
 ('Ireland', 7787),
 ('Hungary', 9316)]


# Exercise 3) Join

Implement the join operator. In SQL it would correspond to the following query:

```sql
SELECT external.*, internal.*
FROM adult as internal INNER JOIN adult as external ON internal.marital-status = external.marital-status
WHERE external.native-country = "Italy" AND internal.native-country = "Ecuador"
```

In [ ]:
def join_map(k,v):
    tupl = k + "," + v

    # Extreiem els atributs 'native-country' i 'marital-status'
    native_country = getAttribute(tupl.split(","), "native-country")
    marital_status = getAttribute(tupl.split(","), "marital-status")

     # Comprovem que 'native-country' és 'Italy' o 'Ecuador'
    if native_country == "Italy" or native_country == "Ecuador":
        # Retornem la columna amb el 'marital_status' com a clau i la tupla (native_country, tupl) com a valor
        # Així fem el inner join amb el 'marital-status'
        return [(marital_status, (native_country, tupl))]
    else:
        return []


def join_reduce(k, lv):
    italy = []
    ecuador = []

    # Separarem les files de 'Italy' i 'Ecuador'
    for country, row in lv:
        if country == "Italy":
            italy.append(row)
        elif country == "Ecuador":
            ecuador.append(row)

    joined_rows = []
    for row_italy in italy:
        for row_ecuador in ecuador:
            joined_rows.append((row_italy, row_ecuador))  # juntem les files d'Ecuador i Italy

    return joined_rows

executeMapReduce(join_map,join_reduce)

[('QxRzsHqMEXkPBFh2hsaavH,47,Self-emp-not-inc,93208,HS-grad,9,Married-civ-spouse,Other-service,Wife,White,Female,0,0,75,Italy,<=50K',
  '6JK5bJqNiM5WXM8fFVA6UQ,50,Private,193884,7th-8th,4,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,Ecuador,<=50K'),
 ('QxRzsHqMEXkPBFh2hsaavH,47,Self-emp-not-inc,93208,HS-grad,9,Married-civ-spouse,Other-service,Wife,White,Female,0,0,75,Italy,<=50K',
  'N4YKpai5fU9ZS2BQTFtWtN,32,Private,227214,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,Other,Male,0,0,40,Ecuador,<=50K'),
 ('QxRzsHqMEXkPBFh2hsaavH,47,Self-emp-not-inc,93208,HS-grad,9,Married-civ-spouse,Other-service,Wife,White,Female,0,0,75,Italy,<=50K',
  'bT8MkoEiuKiSkRMmajTzBE,35,Private,208106,HS-grad,9,Married-civ-spouse,Sales,Husband,White,Male,0,0,40,Ecuador,<=50K'),
 ('QxRzsHqMEXkPBFh2hsaavH,47,Self-emp-not-inc,93208,HS-grad,9,Married-civ-spouse,Other-service,Wife,White,Female,0,0,75,Italy,<=50K',
  'nNvaVZQVaMkKmTcwR6vZKy,90,Private,175491,HS-grad,9,Married-civ-spouse,Craft-